<a href="https://colab.research.google.com/github/SouadSADKI/Forum-App-Go-Backend/blob/master/ENSIAS_SS_1_Code_vulnerability_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import kagglehub
cerengen_securec_c_source_code_for_vulnerabilty_detection_path = kagglehub.dataset_download('cerengen/securec-c-source-code-for-vulnerabilty-detection')

print('Data source import complete.')


100%|██████████| 549M/549M [00:25<00:00, 22.8MB/s]

Extracting files...


Data source import complete.


## Content <a name='toc'></a>
<ol>
<li><a href='#load'> Loading and preparing data </a></li>
<li><a href='#bert'>СodeBert</a></li>
</ol>

In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from tqdm import tqdm, trange
import multiprocessing

from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import torch

import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### 1. Loading and preparing data  <a name='load'></a>

Dataset [SARD](https://www.kaggle.com/datasets/cerengen/securec-c-source-code-for-vulnerabilty-detection/)

Software Assurance Reference Dataset (SARD) is a growing collection of test programs with documented vulnerabilities in the code. Test cases range from small synthetic programs to large applications. The programs are written in C, C++, Java, PHP and C# and cover more than 150 classes of vulnerabilities.

A collection of more than 450,000 examples of vulnerabilities;
Covers over 150 Common Weakness Enumeration (CWE) classes



In [ ]:
pd.set_option('max_colwidth', 400)
df = pd.read_csv(cerengen_securec_c_source_code_for_vulnerabilty_detection_path+'/multi_sard_all_data_marked.csv')
df.head()

,code,CWE-Type,DataType
0,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
1,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
2,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
3,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
4,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD


In [ ]:
unique = len(df['CWE-Type'].unique())
print(f'Number of records in the dataset: {len(df)}')
print(f'CWE types: {unique}')

Number of records in the dataset: 5000
CWE types: 98


In [ ]:
# comment out for faster running
df = df.sample(5000)
unique = len(df['CWE-Type'].unique())
# print(len(df['CWE-Type'].unique()))

In [ ]:
id2label = dict()
label2id = dict()
ind = 0
for i in df['CWE-Type'].unique():
    id2label[ind] = i
    label2id[i] = ind
    ind+=1

In [ ]:
df['label']=df['CWE-Type'].map(label2id)
df.head()

,code,CWE-Type,DataType,label
20087,"\n \n \n #include ""IncludeMarker""\n \n \n static const int intMarker = 5;\n \n #ifndef OMITBAD\n \n \n #endif \n \n #ifndef OMITGOOD\n \n \n \n \n \n \n \n \n \n \n #endif \n \n \n \n #ifdef INCLUDEMAIN\n \n \n #endif\n void VoidFunctionMarker()\n {\n int intMarker;\n \n intMarker = 0;\n if(intMarker==5)\n {\n \n intMarker = INT_MAX;\n }\n if(intMark...",CWE190,SARD,0
43778,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifndef OMITBAD\n \n \n #endif \n \n #ifndef OMITGOOD\n \n \n \n \n #endif \n \n \n \n #ifdef INCLUDEMAIN\n \n \n #endif\n void VoidFunctionMarker()\n {\n char * charPointerMarker;\n charPointerMarker = NULL; \n \n charPointerMarker = (char *)malloc(20*20PointerMarker(char));\n if(globalReturnsTrueOrFalse(...",CWE690,SARD,1
33306,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""advapi32.lib"")\n \n #define DefineMarker 128\n #define DefineMarker (512 / 8)\n \n #define DefineMarker (128 / 8)\n \n #ifndef OMITBAD\n \n \n #endif \n \n #ifndef OMITGOOD\n \n \n \n #endif \n \n \n \n #ifdef INCLUDEMAIN\n \n \n #endif\n void VoidFunctionMarker()\n {\n {\n ...",CWE328,SARD,2
40609,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifndef OMITBAD\n \n \n #endif \n \n #ifndef OMITGOOD\n \n \n \n \n \n \n #endif \n \n \n \n #ifdef INCLUDEMAIN\n \n \n #endif\n void VoidFunctionMarker()\n {\n twoIntsStruct data;\n \n ; \n if(GLOBAL_CONST_TRUE)\n {\n \n \n ; \n }\n }\n\n static void VoidFunctionMarker()\n {\n ...",CWE563,SARD,3
34240,"\n \n \n #include ""IncludeMarker""\n \n #ifndef OMITBAD\n \n \n #endif \n \n #ifndef OMITGOOD\n \n \n \n \n \n \n #endif \n \n \n \n #ifdef INCLUDEMAIN\n \n \n #endif\n void VoidFunctionMarker()\n {\n int intMarker,intMarker;\n int intMarker;\n \n intMarker = -1;\n for(intMarker = 0; intMarker < 1; intMarker++)\n {\n \n intMarker = RAND32();\n }\n ...",CWE369,SARD,4


Split the dataset into training and testing sets

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

### 2. CodeBert Model <a name='bert'></a>

To load the data we will use the [Datasets Library](https://huggingface.co/docs/datasets/index).

Datasets is a library for easily accessing and sharing data sets for audio, computer vision, and natural language processing (NLP) tasks.

Allows you to load data sets and use built-in data processing methods to prepare datasets for use in training neural network models. The library supports the Apache Arrow format and processes large data sets without any memory limitations for optimal speed and efficiency. The library is also integrated with Hugging Face Hub, making it easy to download the dataset and share it with the community.

In [ ]:
dataset = {}
dataset['text'] = list(df_train['code'])
dataset['label'] = list(df_train['label'])
ds = Dataset.from_dict(dataset)
ds = ds.train_test_split(test_size=0.1)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

**Data Collator**

Data Collators are objects that will form batches (portions of data) using a list of data set elements as input. These elements are the same type as train_dataset or eval_dataset elements.

To be able to create batches, data collectors may apply some processing (for example, padding).  

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Evaluate** A library for easily evaluating machine learning models and datasets.

With one line of code, the library gives access to dozens of evaluation methods for various fields (NLP, computer vision, reinforcement learning, etc.)

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

[CodeBert](https://github.com/microsoft/CodeBERT)

Pre-trained model taken from a Microsoft Research paper. The CodeSearchNet dataset was used to train this model, but instead of generating comments, it was used to train a RoBERTa-based model on human-readable representations of code and natural language.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base-mlm", num_labels=unique, id2label=id2label, label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 class The Trainer provides an API for full-featured training in PyTorch for most common use cases.

Before instantiating a Trainer, you must create arguments for training to access all customization points during training.

The API supports distributed training on multiple GPUs, mixed precision with NVIDIA Apex, and a native accelerator for PyTorch.

Trainer contains a basic training loop that supports the above features. To implement custom functions, you can override the following methods:
- get_train_dataloader — Creates the training DataLoader.
- get_eval_dataloader — Creates the evaluation DataLoader.
- get_test_dataloader — Creates the test DataLoader.
- log — Logs information on the various objects watching training.
- create_optimizer_and_scheduler — Sets up the optimizer and learning rate scheduler if they were not passed at - init. Note, that you can also subclass or override the create_optimizer and create_scheduler methods separately.
- create_optimizer — Sets up the optimizer if it wasn’t passed at init.
- create_scheduler — Sets up the learning rate scheduler if it wasn’t passed at init.
- compute_loss - Computes the loss on a batch of training inputs.
- training_step — Performs a training step.
- prediction_step — Performs an evaluation/test step.
- evaluate — Runs an evaluation loop and returns metrics.
- predict — Returns predictions (with metrics if labels are available) on a test set.

In [ ]:
training_args = TrainingArguments(
    output_dir="trained_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-31-021e2f86ab0a>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.877321,0.320000


TrainOutput(global_step=225, training_loss=3.421096462673611, metrics={'train_runtime': 382.2538, 'train_samples_per_second': 9.418, 'train_steps_per_second': 0.589, 'total_flos': 948016233676800.0, 'train_loss': 3.421096462673611, 'epoch': 1.0})

### Evaluating the trained model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

preds = []
for i in df_test['code'].values:
    with torch.no_grad():
        inputs = tokenizer(i, return_tensors="pt",  truncation=True).to(device)
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        preds.append(predicted_class_id)

In [ ]:
y_true = [id2label[i] for i in df_test['label'].values]
y_pred = [id2label[i] for i in preds]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

      CWE114       0.00      0.00      0.00        13
      CWE121       0.24      0.82      0.37       115
      CWE122       0.00      0.00      0.00        67
      CWE124       0.00      0.00      0.00        32
      CWE126       0.00      0.00      0.00        20
      CWE127       0.00      0.00      0.00        37
      CWE134       0.47      0.84      0.60        57
      CWE188       0.00      0.00      0.00         1
      CWE190       0.29      0.98      0.45        96
      CWE191       0.06      0.03      0.04        62
      CWE194       0.00      0.00      0.00        24
      CWE195       0.00      0.00      0.00        26
      CWE197       0.00      0.00      0.00        14
      CWE222       0.00      0.00      0.00         1
      CWE223       0.00      0.00      0.00         1
      CWE226       0.00      0.00      0.00         1
      CWE244       0.00      0.00      0.00         2
      CWE252       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
